In [6]:
# notebooks/analyze.ipynb
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("StockSentimentAnalysis") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/stock_db.processed_news") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

# Tải dữ liệu đã xử lý
df = spark.read.format("mongo").load()

# Đếm số lượng tin tức
total_articles = df.count()
print(f"Tổng số bài viết: {total_articles}")

# Phân loại sentiment
sentiment_counts = df.groupBy("sentiment").count().toPandas()
sentiment_counts['sentiment'] = sentiment_counts['sentiment'].map({1: 'Tích cực', 0: 'Tiêu cực'})

# Trực quan hóa
plt.figure(figsize=(8, 6))
sns.barplot(x='sentiment', y='count', data=sentiment_counts)
plt.title("Phân bố Sentiment của tin tức chứng khoán")
plt.xlabel("Sentiment")
plt.ylabel("Số lượng bài viết")
plt.show()

# Dừng SparkSession
spark.stop()

: 